In [79]:
#패키지 로드 
import pandas as pd
import glob
import os
import numpy as np
import re
import time
from tqdm import tqdm_notebook as tqdm
import sys
from konlpy.tag import Okt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from ckonlpy.tag import Twitter
from ckonlpy.tag import Postprocessor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import sklearn as skl
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

In [62]:
data =  pd.read_excel('C:/Users/USER/Desktop/labelingdata2018_word.xlsx', index_col=0)#데이터 불러오기
data['words'] = [i.replace("'", "").replace('[', '').replace(']', '').replace(' ', '').split(',') for i in
          list(data['words'])]  # words가 텍스트 형식으로 되어 있을 경우
data = data.drop_duplicates(['text'])
data = pd.DataFrame(data).reset_index(drop=True)

In [13]:
from sklearn.cluster import KMeans
# common_text에는 파싱된 워드 리스트들이 들어가 있음.
from gensim.test.utils import common_texts
# Doc2Vec이 우리가 텍스트를 사용해서 학습되는 모델(뉴럴넷)이고 
# TaggedDocument가 넘겨주는 텍스트들. 
# 여기서, corpus와 ID들을 함께 넘겨줘야 하는데, 여기서 ID는 tag와 같은 말임
# Q: 여기서, 여러 tag를 함께 넘겨준 다음, 적합한 태그를 찾아주는 방식으로도 처리할 수 있는지 파악하는 것이 필요함. 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [30]:
common_texts_and_tags = [
    (text, [f"str_{i}",]) for i, text in enumerate(list(data['text']))
]

In [31]:
# text: 단어별로 분할되어 있는 리스트 
# tags: 해당 문서를 의미하는 태그. 여기서, tags를 unique id로 넣기는 했는데, 그렇지 않게 넣어서 학습시키는 것도 가능할 것으로 보임. 
TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in common_texts_and_tags]
# training. 

In [81]:

# 여러 Parameter들을 사용하여 튜닝할 수 있음. 
model = Doc2Vec(TRAIN_documents, vector_size=100, window=5, epochs=100, min_count=75, workers=4)
print("##"*20)
print("Accessing Document vector")
print("##"*20)

########################################
Accessing Document vector
########################################


In [82]:
vecdata = pd.DataFrame(columns=['x{}'.format(i) for i in range(100)])
for i in range(len(model.docvecs)):
    vecdata.loc[i] = model.docvecs[i]

In [83]:
vecdata.head(-5)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99
0,-0.685598,-0.566988,-0.717150,1.514102,2.972272,-0.086815,-0.280427,1.174188,2.082348,0.803516,...,-2.760795,0.431229,-0.275619,0.159013,-1.135924,2.496452,0.806691,0.440451,-1.043214,-1.467086
1,-1.931718,-1.759627,0.342644,1.419052,2.034276,0.967912,0.977340,0.320818,0.698538,0.314645,...,-1.374371,1.402329,-1.574545,0.060470,-0.546270,1.606009,1.378197,0.918937,-0.694451,-1.129756
2,1.576509,-2.761534,-0.942052,0.116545,3.130720,-2.270481,-2.368831,-1.604045,0.221741,-0.620816,...,-1.282152,-0.964323,-0.561131,-2.582947,0.093865,1.457027,-2.676169,-0.150945,-2.241054,0.810656
3,-1.255068,0.122275,1.618620,-2.510324,1.400617,0.022608,-0.153872,-0.398147,2.740014,1.764013,...,-1.889227,0.656482,0.294138,-2.245999,0.427598,-1.688722,-0.644998,0.429197,-1.279981,-0.071176
4,-0.079214,-1.129157,1.231191,-0.566248,1.373620,-0.379773,0.143489,0.191982,3.873324,1.712120,...,-1.549306,-0.469485,-2.660117,-2.614763,1.686812,0.307204,0.027828,0.247240,0.525728,0.077440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23920,0.553472,-1.978945,1.977563,0.667062,2.272826,0.105988,-1.478042,-0.798892,3.002175,0.541601,...,-3.429466,0.157460,1.008126,0.101187,1.592729,-1.420717,0.242745,-1.699471,-0.054595,0.602049
23921,0.678571,0.528751,-2.716191,-1.623171,2.616068,1.085269,-3.093103,1.613357,2.977688,4.679352,...,0.586345,1.994864,-0.590486,-2.572109,-0.821871,-2.267176,0.066805,-2.903670,-0.698696,0.650518
23922,-0.975698,-0.855860,-1.898499,-0.936282,1.640856,0.909821,0.422433,-2.898981,2.321481,5.234810,...,-2.140605,1.097448,1.882155,-1.916697,-2.830283,-0.881707,-1.266528,-2.172483,2.535764,2.519199
23923,-0.284502,-1.391590,-0.778774,0.253783,0.884285,-0.349884,-1.350356,0.204208,2.411634,4.786092,...,-0.715728,-0.026798,1.925280,1.135894,-1.014811,-0.313248,-1.429242,-4.461747,0.022419,1.439779


In [84]:
data_doc2vec = pd.concat([data,vecdata],axis = 1)
data_doc2vec.head(-5)

,date,title,text,user1,user2,user3,user4,user5,user6,sum,...,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99
0,20180102,"민주 ""금주중 개헌 정개 사개특위 구성해야""",(서울=연합뉴스) 서혜림 기자 = 더불어민주당 박홍근 원내수석부대표는 2일 이번 주...,0,1,0,0,0,1,2,...,-2.760795,0.431229,-0.275619,0.159013,-1.135924,2.496452,0.806691,0.440451,-1.043214,-1.467086
1,20180102,"민주 ""금주중 개헌 정개 사개특위 구성해야""",더불어민주당 박홍근 원내수석부대표는 오늘(2일) 헌법개정 및 정치개혁특위를 이번 주...,0,1,0,0,0,1,2,...,-1.374371,1.402329,-1.574545,0.060470,-0.546270,1.606009,1.378197,0.918937,-0.694451,-1.129756
2,20180102,일자리가 풍족한 한 해를 기원하며,(서울=연합뉴스) 안정원 기자 = 이낙연 국무총리가 김동연 경제부총리_ 기획재정부 ...,1,0,0,0,1,1,3,...,-1.282152,-0.964323,-0.561131,-2.582947,0.093865,1.457027,-2.676169,-0.150945,-2.241054,0.810656
3,20180102,한국당 일자리 정책은 연목구어( ),-반기업 정책에 일자리 늘기 바라는 모순 [헤럴드경제=홍태화 기자] 자유한국당이 문...,1,0,0,0,1,1,3,...,-1.889227,0.656482,0.294138,-2.245999,0.427598,-1.688722,-0.644998,0.429197,-1.279981,-0.071176
4,20180102,김태년 주거 의료 교육 등 5대 생활비 획기적으로 줄일 것,- 적폐청산 은 국민의 요구이자 시대 흐름 [헤럴드경제=최진성 기자] 김태년 더불어...,0,0,0,0,0,0,0,...,-1.549306,-0.469485,-2.660117,-2.614763,1.686812,0.307204,0.027828,0.247240,0.525728,0.077440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23920,20181231,"[신년사] 홍남기 ""경제활력 제고 역점 사회적 대타협의 원년""",(세종=연합뉴스) 이 율 기자 = 홍남기 부총리 겸 기획재정부 장관은 31일 내년 ...,0,0,0,0,0,0,0,...,-3.429466,0.157460,1.008126,0.101187,1.592729,-1.420717,0.242745,-1.699471,-0.054595,0.602049
23921,20181231,"최종구 ""과중한 검사 제재가 혁신 발목 금융감독 행태 개선""",[신년사] 금소법·통합감독법·지배구조법 등 입법 강조 전 업권 DSR 도입·상시 시...,0,0,0,0,0,0,0,...,0.586345,1.994864,-0.590486,-2.572109,-0.821871,-2.267176,0.066805,-2.903670,-0.698696,0.650518
23922,20181231,[은행장 전망] 최대 이슈는 리스크 관리 주담대 감소 전망도,"""경기 둔화에 규제 강화…대출금리 상승 불가피"" 내년 가계대출 증가율 대체로 4%대...",0,0,0,0,0,0,0,...,-2.140605,1.097448,1.882155,-1.916697,-2.830283,-0.881707,-1.266528,-2.172483,2.535764,2.519199
23923,20181231,[은행장 전망] 올해 3천명 뽑은 은행권 내년에도 시중은행 채용문 '활짝',"""점포 수 축소 불가피""…특별퇴직·노사갈등으로 연말 추가 퇴직은 없을 듯 (서울=연...",0,0,1,0,0,0,1,...,-0.715728,-0.026798,1.925280,1.135894,-1.014811,-0.313248,-1.429242,-4.461747,0.022419,1.439779


In [64]:
data_doc2vec.to_excel('C:/Users/USER/Desktop/labelingdata2018_word_d2v.xlsx')

In [77]:
inferred_v = model.infer_vector(['고용'])
inferred_v
model.docvecs.most_similar([inferred_v],topn=3)

[('str_17086', 0.3561972379684448),
 ('str_17083', 0.3541346490383148),
 ('str_1817', 0.34969010949134827)]

In [85]:
#2번모델 knn돌려보기
x = data_doc2vec.iloc[:,12:112]
y = data_doc2vec.iloc[:,10]

x_train, x_test, y_train, y_test = skl.model_selection.train_test_split(x, y,random_state=100,stratify=y) #stratify = y y를 동일한 비율로 나누어 준다.
#정규화시키는 함수
scaler = skl.preprocessing.StandardScaler()

#x_train 정규화
scaler.fit(x_train) #x_train 데이터에 대해서 평균과 표준편차 계산
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
###########knn사용해서 분류해보기

classifier = KNeighborsClassifier(n_neighbors= 2)
classifier.fit(x_train,y_train)
knn_pre = classifier.predict(x_test)

#성능 평가
print(confusion_matrix(y_test,knn_pre))

#평가결과 확인
print(classification_report(y_test,knn_pre)) #'''
print(classifier.score(x_test, y_test))

[[5115  261]
 [ 405  202]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      5376
           1       0.44      0.33      0.38       607

    accuracy                           0.89      5983
   macro avg       0.68      0.64      0.66      5983
weighted avg       0.88      0.89      0.88      5983

0.8886846063847568
